# Conversion between different notations

In [1]:
import numpy as np
import math

## Tests

In [25]:
e2 = ( -np.arctan(0.25), -np.arcsin(8.0/9.0), np.arctan(4.0))
print(e2)
A = np.array([[0.111111 , -0.888889, -0.444444], 
              [0.444444 ,  0.444444, -0.777778],
              [0.888889 , -0.111111,  0.444444]])

Aa = ([1/3, -2/3, 2/3], math.pi/2)
q = (1/(3 * math.sqrt(2))  , -math.sqrt(2)/3 , math.sqrt(2)/3, 1/math.sqrt(2))

(-0.24497866312686414, -1.09491407713448, 1.3258176636680326)


$ Matrix$ $notation \leftrightarrow Euler's $ $ angles $

In [3]:
def conversionFromMatrixToEulers(transformation):
    if abs(transformation[2,0]) < 1:
        # normal case
        yaw = math.atan2(transformation[1,0], transformation[0,0])
        pitch = np.arcsin(-transformation[2,0])
        rolling = math.atan2( transformation[2,1], transformation[2,2])
    elif transformation[2,0] == -1:
        # gimball lock
        yaw = math.atan2( transformation[1,1], -transformation[0,1])
        pitch = math.pi / 2
        rolling = 0
    elif transformation[2,0] == 1:
        # gimball lock
        yaw = math.atan2( transformation[0,1], -transformation[1,1])
        pitch = -math.pi / 2
        rolling = 0
    
    return (yaw, pitch, rolling)

In [4]:
conversionFromMatrixToEulers(A)

(1.3258176636680326, -1.0949143196701623, -0.24497866312686414)

In [5]:
# normal test - without gimball lock
a = np.array([1, -8 , 4, 4, 4, -7, 8, -1, 4]).reshape(3,3)/9
a

array([[ 0.11111111, -0.88888889,  0.44444444],
       [ 0.44444444,  0.44444444, -0.77777778],
       [ 0.88888889, -0.11111111,  0.44444444]])

In [6]:
conversionFromMatrixToEulers(a)

(1.3258176636680326, -1.09491407713448, -0.24497866312686414)

In [7]:
q = 1/math.sqrt(2)
b = np.array([0,q,q,0,q,-q,-1,0,0]).reshape(3,3)
b

array([[ 0.        ,  0.70710678,  0.70710678],
       [ 0.        ,  0.70710678, -0.70710678],
       [-1.        ,  0.        ,  0.        ]])

In [8]:
conversionFromMatrixToEulers(b)

(2.356194490192345, 1.5707963267948966, 0)

In [9]:
def conversionFromEulersToMatrix(EulerAngles):
    cs_rolling = math.cos(EulerAngles[0])
    sn_rolling = math.sin(EulerAngles[0])
    
    cs_pitch = math.cos(EulerAngles[1])
    sn_pitch = math.sin(EulerAngles[1])
    
    cs_yaw = math.cos(EulerAngles[2])
    sn_yaw = math.sin(EulerAngles[2])
    
    RZ = np.array([cs_yaw, -sn_yaw, 0,
                   sn_yaw, cs_yaw,  0,
                   0     , 0,       1]).reshape(3,3)
    
    
    RY = np.array([cs_pitch, 0, sn_pitch,
                  0        , 1,        0,
                  -sn_pitch, 0, cs_pitch]).reshape(3,3)
    
    
    RX = np.array([1       ,   0,           0,
                   0, cs_rolling, -sn_rolling,
                   0, sn_rolling,  cs_rolling]).reshape(3,3)
    
    P = np.dot(RZ,RY)
    P = np.dot(P,RX)
    return P

In [10]:
conversionFromEulersToMatrix(e2)

array([[ 0.11111111, -0.88888889, -0.44444444],
       [ 0.44444444,  0.44444444, -0.77777778],
       [ 0.88888889, -0.11111111,  0.44444444]])

In [11]:
A

array([[ 0.111111, -0.888889, -0.444444],
       [ 0.444444,  0.444444, -0.777778],
       [ 0.888889, -0.111111,  0.444444]])

In [12]:
conversionFromEulersToMatrix((math.pi/4, 0.5, math.pi/6))

array([[ 0.76000879, -0.05996641,  0.64714038],
       [ 0.43879128,  0.78187496, -0.44286991],
       [-0.47942554,  0.62054458,  0.62054458]])

$Matrix$ $notation \leftrightarrow Angle$ $Axis$

In [13]:
def A2AngleAxis(transformation):
    l, x = np.linalg.eig(transformation)
    p = []
    
    for i in range(0,len(l)):
        # print(eigs[0][i], eigs[1][i])
        if (l[i].real-1.0) < 0.2:
            p = x[i]
            p = [e.real for e in p]
            print(p)
            break
    
    euc = math.sqrt(p[0]**2 + p[1]**2 + p[2]**2)
    p = [e/euc for e in p]
    u = np.array([-p[1], p[0], 0])
    euc = math.sqrt((u[0]**2 + u[1]**2 + u[2]**2))
    u = np.array([e/euc for e in u])
    u = u.reshape(3,1)
    u_prim = np.dot(transformation, u)
    u_prim = u_prim.reshape(3,)
    
    u = u.reshape(1,3)
    u = u[0]
    
    angle = np.arccos(np.dot(u,u_prim))
    
    a = np.array([u[0], u[1], u[2],
                  u_prim[0], u_prim[1], u_prim[2],
                  p[0], p[1], p[2]], dtype = 'float').reshape(3,3)
    
    if np.linalg.det(a) < 0:
        p = (-1) * p
    
    return (p, angle)
    

In [14]:
A2AngleAxis(A)

[-0.6666665925925579, -0.6666665925925579, 0.33333362962961616]


([-0.6666665925925589, -0.6666665925925589, 0.33333362962961666],
 1.0471975511965976)

In [15]:
Aa

([0.3333333333333333, -0.6666666666666666, 0.6666666666666666],
 1.5707963267948966)

In [16]:
def Rodriguez(angle, vector):
    p1 = vector[0]
    p2 = vector[1]
    p3 = vector[2]
    
    q = math.sqrt(p1**2 + p2 ** 2 + p3**2)
    p1 /= q
    p2 /= q
    p3 /= q
    p = np.array([p1,p2,p3]).reshape(1,3)
    pT = p.reshape(3,1)
    
    E = np.eye(3)
    px = np.array([0, -p3, p2, p3, 0, -p1, -p2, p1, 0]).reshape(3,3)
    
    A = math.sin(angle) * px
    A = A + E * math.cos(angle)
    A = A + (1 - math.cos(angle)) * pT.dot(p)
    
    return A
print(A)

[[ 0.111111 -0.888889 -0.444444]
 [ 0.444444  0.444444 -0.777778]
 [ 0.888889 -0.111111  0.444444]]


In [17]:
Rodriguez(vector = Aa[0], angle = Aa[1])

array([[ 0.11111111, -0.88888889, -0.44444444],
       [ 0.44444444,  0.44444444, -0.77777778],
       [ 0.88888889, -0.11111111,  0.44444444]])

In [18]:
p = np.array([1,1,1]) * (math.sqrt(3) / 3)
Rodriguez(math.pi/3, p)

array([[ 0.66666667, -0.33333333,  0.66666667],
       [ 0.66666667,  0.66666667, -0.33333333],
       [-0.33333333,  0.66666667,  0.66666667]])

$Angle$ $axis \leftrightarrow Quaternions $ 

In [19]:
def AngleAxis2Q(angle, vector):
    w = math.cos(angle/2)
    p_intensity = math.sqrt(vector[0]**2 + vector[1]**2 + vector[2]**2)
    p = np.array([vector[0], vector[1], vector[2]])/p_intensity
    
    qijk = math.sin(angle/2) * p
    
    return (qijk[0], qijk[1], qijk[2], w);   

In [28]:
print(AngleAxis2Q(vector = Aa[0], angle = Aa[1]))
q

(0.2357022603955158, -0.4714045207910316, 0.4714045207910316, 0.7071067811865476)


(0.2357022603955158,
 -0.47140452079103173,
 0.47140452079103173,
 0.7071067811865475)

In [26]:
def Q2AngleAxis(q):
    i, j, k , w = q
    
    if w < 0:
        i *= -1
        j *= -1
        k *= -1
        w *= -1
    
    angle = 2 * np.arccos(w)
    
    if abs(w) == 1:
        p = (1,0,0)
    else:
        u = math.sqrt(i**2 + j**2 + k**2)
        p = (i/u, j/u, k/u)
    
    return (p,angle)
        

In [27]:
print(Q2AngleAxis(q))
Aa

((0.33333333333333326, -0.6666666666666667, 0.6666666666666667), 1.5707963267948968)


([0.3333333333333333, -0.6666666666666666, 0.6666666666666666],
 1.5707963267948966)